In [ ]:
# General
import numpy as np
import pandas as pd
import nltk
import random
import os
from os import path
os.chdir('/kaggle/input/')
from PIL import Image

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS

# Set Plot Theme
sns.set_palette([
    "#30a2da",
    "#fc4f30",
    "#e5ae38",
    "#6d904f",
    "#8b8b8b",
])
# Alternate # plt.style.use('fivethirtyeight')

# Pre-Processing
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

# Modeling
import statsmodels.api as sm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk.util import ngrams
from collections import Counter
from gensim.models import word2vec

# Warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.drop(data.columns[0],inplace=True,axis=1)

In [ ]:
data.isna().sum()

In [ ]:
data['Title']=data['Title'].fillna('c')
data['Review Text']=data['Review Text'].fillna('c')
data['Division Name']=data['Division Name'].fillna('c')
data['Department Name']=data['Department Name'].fillna('c')
data['Class Name']=data['Class Name'].fillna('c')

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
#word length
data['word_count']=data['Review Text'].str.split().apply(len)

In [ ]:
#character length
data['charac_count']=data['Review Text'].apply(len)

In [ ]:
# Boolean for Positive and Negative Reviews
data["Label"] = 0 #negative
data.loc[data.Rating >= 3,["Label"]] = 1 #positive

In [ ]:
data.sample(30)

In [ ]:
unique_count = []
for x in data.columns:
    unique_count.append([x,len(data[x].unique()),data[x].isnull().sum()])

In [ ]:
print("Dataframe Dimension: {} Rows, {} Columns".format(*data.shape))
pd.DataFrame(unique_count, columns=["Column","Unique","Missing"]).set_index("Column").T

In [ ]:
data.describe().T.drop("count",axis=1)

In [ ]:
data[["Title", "Division Name","Department Name","Class Name"]].describe().T.drop("count",axis=1)

In [ ]:
# Continous Distributions
f, ax = plt.subplots(1,3,figsize=(12,4), sharey=False)
sns.distplot(data.Age, ax=ax[0])
ax[0].set_title("Age Distribution")
ax[0].set_ylabel("Density")
sns.distplot(data["Positive Feedback Count"], ax=ax[1])
ax[1].set_title("Positive Feedback Count Distribution")
sns.distplot(np.log10((data["Positive Feedback Count"][data["Positive Feedback Count"].notnull()]+1)), ax=ax[2])
ax[2].set_title("Positive Feedback Count Distribution\n[Log 10]")
ax[2].set_xlabel("Log Positive Feedback Count")
plt.tight_layout()
plt.show()

In [ ]:
# Percentage Accumulation from "Most Wealthy"
def percentage_accumulation(series, percentage):
    return (series.sort_values(ascending=False)
            [:round(series.shape[0]*(percentage/100))]
     .sum()/series
     .sum()*100)
# Gini Coefficient- Inequality Score
# Source: https://planspace.org/2013/06/21/how-to-calculate-gini-coefficient-from-raw-data-in-python/
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2.
    return (fair_area - area) / fair_area
# Cumulative Percentage of Positive Feedback assigned Percent of Reviewers (from most wealthy)
inequality = []
for x in list(range(100)):
    inequality.append(percentage_accumulation(data["Positive Feedback Count"], x))

# Generic Matplotlib Plot
plt.plot(inequality)
plt.title("Percentage of Positive Feedback by Percentage of Reviews")
plt.xlabel("Review Percentile starting with Feedback")
plt.ylabel("Percent of Positive Feedback Received")
plt.axvline(x=20, c = "r")
plt.axvline(x=53, c = "g")
plt.axhline(y=78, c = "y")
plt.axhline(y=100, c = "b", alpha=.3)
plt.show()

# 80-20 Rule Confirmation
print("{}% of Positive Feedback belongs to the top 20% of Reviews".format(
    round(percentage_accumulation(data["Positive Feedback Count"], 20))))

# Gini
print("\nGini Coefficient: {}".format(round(gini(data["Positive Feedback Count"]),2)))

In [ ]:
# Cumulative Percentage of Positive Feedback assigned Percent of Reviewers (from most wealthy)
top_20 = data["Positive Feedback Count"].sort_values(ascending=False)[:round(data.shape[0]*(20/100))]

inequality = []
for x in list(range(100)):
    inequality.append(percentage_accumulation(top_20, x))

# Generic Matplotlib Plot
plt.plot(inequality)
plt.title("Percentage of Positive Feedback by Percentage of Reviews")
plt.xlabel("Review Percentile starting with Feedback")
plt.ylabel("Percent of Positive Feedback Received")
plt.axvline(x=20, c = "r")
plt.axhline(y=47, c = "r")
plt.axhline(y=100, c = "b", alpha=.3)

plt.show()

# 80-20 Rule Confirmation
print("{}% of Positive Feedback belongs to the top 20% of Reviews".format(
    round(percentage_accumulation(top_20, 20))))

# Gini
print("\nGini Coefficient: {}".format(round(gini(top_20),2)))

In [ ]:
row_plots=["Division Name","Department Name"]
f,axes=plt.subplots(1,len(row_plots),figsize=(14,4),sharex=False)
for count,eleinrow in enumerate(row_plots):
    sns.countplot(y=eleinrow,data=data,order=data[eleinrow].value_counts().index,ax=axes[count])
    axes[count].set_title("Count of Categories in {}".format(eleinrow))
    axes[count].set_xlabel("")
    axes[count].set_xlabel("Frequency Count")
axes[0].set_ylabel("Category")
axes[1].set_ylabel("")
plt.show()


In [ ]:
# Clothing ID Category
f, axes = plt.subplots(1,1, figsize=[14,7])
num=30
sns.countplot(y=data["Clothing ID"],data=data[data["Clothing ID"].isin(data['Clothing ID'].value_counts()[:num].index)],
              order=data['Clothing ID'].value_counts()[:num].index)
axes[0].set_title("Frequency Count of Clothing ID\nTop 30")
axes[0].set_xlabel("Count")

In [ ]:
data[data['Clothing ID'].isin([1078,862,1094])].describe().T.drop('count',axis=1)

For the top 3 selling cloth id's,average age of buyers is around 43 and the average rating they give is 4.

In [ ]:
data.loc[data["Clothing ID"].isin([1078,862,1094]),["Title", "Division Name","Department Name","Class Name"]]

In [ ]:
data.loc[data["Clothing ID"].isin([1078,862,1094]),["Title", "Division Name","Department Name","Class Name"]].describe().T.drop('count',axis=1)

In [ ]:
sns.countplot(y="Class Name",data=data,order=data['Class Name'].value_counts().index)

In [ ]:
cat_dtypes=['Rating','Recommended IND','Label']
f,axes=plt.subplots(1,len(cat_dtypes),figsize=(14,4),sharex=False)
increment=0
for i in range(len(cat_dtypes)):
    sns.countplot(x=cat_dtypes[increment],data=data,order=data[cat_dtypes[increment]].value_counts().index,ax=axes[i])
    axes[i].set_title("Frequency Distribution for\n{}".format(cat_dtypes[increment]))
    axes[i].set_ylabel("Occurrence")
    axes[i].set_xlabel("{}".format(cat_dtypes[increment]))
    increment += 1
axes[1].set_ylabel("")
axes[2].set_ylabel("")
plt.show()

There are higher number of positive ratings. Recommendations are positive i.e. the items that have positive reviews are recommended.

In [ ]:
f,axes=plt.subplots(2,4,figsize=(17,8),sharex=False)
for k,xvar in enumerate(['word_count','charac_count']):
    for i,y in enumerate(['Rating','Department Name','Recommended IND']):
        for x in set(data[y][data[y].notnull()]):
            sns.kdeplot(data[xvar][data[y]==x],label=x, shade=False, ax=axes[k,i])
            if k is 0:
                axes[k,i].set_title('{} Distribution (X)\nby {}'.format(xvar, y))
            else:
                axes[k,i].set_title('For {} (X)'.format(xvar))
    axes[k,0].set_ylabel('Occurrence Density')
    axes[k,i].set_xlabel('')

In [ ]:
data[['Rating','Department Name','Recommended IND','word_count','charac_count']].describe()

In [ ]:
f, ax = plt.subplots(1,2,figsize=(10, 7), sharey=True)
fsize = 13
sns.heatmap(pd.crosstab(data['Class Name'], data["Department Name"], normalize = 'columns').mul(100).round(0)
            ,annot=True, fmt="g", linewidths=.5, ax=ax[0],cbar=False,cmap="Blues")
ax[0].set_title('Class Name Count by Count - Crosstab\nHeatmap % Distribution by Column', fontsize = fsize)
ax[1] = sns.heatmap(pd.crosstab(data['Class Name'], data["Department Name"], normalize = 'index').mul(100).round(0)
            ,annot=True, fmt="2g", linewidths=.5, ax=ax[1],cmap="Blues",
                cbar_kws={'label': 'Percentage %'})
ax[1].set_title('Class Name Count by Count - Crosstab\nHeatmap % Distribution by Index', fontsize = fsize)
ax[1].set_ylabel('')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# Pre-Processing
SIA = SentimentIntensityAnalyzer()
data["Review Text"]= data["Review Text"].astype(str)

# Applying Model, Variable Creation
data['Polarity Score']=data["Review Text"].apply(lambda x:SIA.polarity_scores(x)['compound'])
data['Neutral Score']=data["Review Text"].apply(lambda x:SIA.polarity_scores(x)['neu'])
data['Negative Score']=data["Review Text"].apply(lambda x:SIA.polarity_scores(x)['neg'])
data['Positive Score']=data["Review Text"].apply(lambda x:SIA.polarity_scores(x)['pos'])

# Converting 0 to 1 Decimal Score to a Categorical Variable
data['Sentiment']=''
data.loc[data['Polarity Score']>0,'Sentiment']='Positive'
data.loc[data['Polarity Score']==0,'Sentiment']='Neutral'
data.loc[data['Polarity Score']<0,'Sentiment']='Negative'

In [ ]:
def percentstandardize_barplot(x,y,hue, data, ax=None, order= None):
    """
    Standardize by percentage the data using pandas functions, then plot using Seaborn.
    Function arguments are and extention of Seaborns'.
    """
    sns.barplot(x= x, y=y, hue=hue, ax=ax, order=order,
    data=(data[[x, hue]]
     .reset_index(drop=True)
     .groupby([x])[hue]
     .value_counts(normalize=True)
     .rename('Percentage').mul(100)
     .reset_index()
     .sort_values(hue)))
    plt.title("Percentage Frequency of {} by {}".format(hue,x))
    plt.ylabel("Percentage %")

In [ ]:
huevar = "Recommended IND"
xvar = "Sentiment"
f, axes = plt.subplots(1,2,figsize=(12,5))
sns.countplot(x=xvar, hue=huevar,data=data, ax=axes[0], order=["Negative","Neutral","Positive"])
axes[0].set_title("Occurence of {}\nby {}".format(xvar, huevar))
axes[0].set_ylabel("Count")
percentstandardize_barplot(x=xvar,y="Percentage", hue=huevar,data=data, ax=axes[1])
axes[1].set_title("Percentage Normalized Occurence of {}\nby {}".format(xvar, huevar))
axes[1].set_ylabel("% Percentage by {}".format(huevar))
plt.show()

In [ ]:
f, axes = plt.subplots(2,2, figsize=[9,9])
sns.countplot(x="Sentiment", data=data, ax=axes[0,0], order=["Negative","Neutral","Positive"])
axes[0,0].set_xlabel("Sentiment")
axes[0,0].set_ylabel("Count")
axes[0,0].set_title("Overall Sentiment Occurrence")

sns.countplot(x="Rating", data=data, ax=axes[0,1])
axes[0,1].set_xlabel("Rating")
axes[0,1].set_ylabel("")
axes[0,1].set_title("Overall Raiting Occurrence")

percentstandardize_barplot(x="Rating",y="Percentage",hue="Sentiment",data=data, ax=axes[1,0])
axes[1,0].set_xlabel("Rating")
axes[1,0].set_ylabel("Percentage %")
axes[1,0].set_title("Standardized Percentage Raiting Frequency\nby Sentiment")

percentstandardize_barplot(x="Sentiment",y="Percentage",hue="Rating",data=data, ax=axes[1,1])
axes[1,1].set_ylabel("Occurrence Frequency")
axes[1,1].set_title("Standardized Percentage Sentiment Frequency\nby Raiting")
axes[1,1].set_xlabel("Sentiment")
axes[1,1].set_ylabel("")

f.suptitle("Distribution of Sentiment Score and Rating for Customer Reviews", fontsize=14)
f.tight_layout()
f.subplots_adjust(top=0.92)
plt.show()



In [ ]:
# Tweakable Variables (Note to Change Order Arguement if Xvar is changed)
xvar = "Sentiment"
huevar = "Department Name"
rowvar = "Recommended IND"

# Plot
f, axes = plt.subplots(2,2,figsize=(10,10), sharex=False,sharey=False)
for i,x in enumerate(set(data[rowvar][data[rowvar].notnull()])):
    percentstandardize_barplot(x=xvar,y="Percentage", hue=huevar,data=data[data[rowvar] == x],
                 ax=axes[i,0], order=["Negative","Neutral","Positive"])
    percentstandardize_barplot(x=xvar,y="Percentage", hue="Rating",data=data[data[rowvar] == x],
                 ax=axes[i,1], order=["Negative","Neutral","Positive"])

# Plot Aesthetics
axes[1,0].legend_.remove()
axes[1,1].legend_.remove()
axes[0,1].set_ylabel("")
axes[1,1].set_ylabel("")
axes[0,0].set_xlabel("")
axes[0,1].set_xlabel("")
axes[0,0].set_ylabel("Recommended = FALSE\nPercentage %")
axes[1,0].set_ylabel("Recommended = TRUE\nPercentage %")
axes[1,1].set_title("")

# Common title and ylabel
f.text(0.0, 0.5, 'Subplot Rows\nSliced by Recommended', va='center', rotation='vertical', fontsize=12)
f.suptitle("Review Sentiment by Department Name and Raiting\nSubplot Rows Slice Data by Recommended", fontsize=16)
f.tight_layout()
f.subplots_adjust(top=0.93)
plt.show()

In [ ]:
# Plot Correlation Matrix
f, ax = plt.subplots(figsize=[9,6])
ax = sns.heatmap(data.corr(), annot=True,
                 fmt=".2f",cbar_kws={'label': 'Correlation Coefficient'})
ax.set_title("Correlation Matrix for All Variables")
plt.show()

# Sentiment Positivity Score by Positive Feedback Count
ax = sns.jointplot(x= data["Positive Feedback Count"], y=data["Positive Score"], kind='reg', color='r')
plt.show()

In [ ]:
stopwords = set(STOPWORDS)
size = (10,7)

def cloud(text, title, stopwords=stopwords, size=size):
    """
    Function to plot WordCloud
    Includes: 
    """
    # Setting figure parameters
    mpl.rcParams['figure.figsize']=(10.0,10.0)
    mpl.rcParams['font.size']=12
    mpl.rcParams['savefig.dpi']=100
    mpl.rcParams['figure.subplot.bottom']=.1 
    
    # Processing Text
    # Redundant when combined with my Preprocessing function
    wordcloud = WordCloud(width=1600, height=800,
                          background_color='black',
                          stopwords=stopwords,
                         ).generate(str(text))
    
    # Output Visualization
    fig = plt.figure(figsize=size, dpi=80, facecolor='k',edgecolor='k')
    plt.imshow(wordcloud,interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=50,color='y')
    plt.tight_layout(pad=0)
    plt.show()
    
# Frequency Calculation [One-Gram]
def wordfreqviz(text, x):
    word_dist = nltk.FreqDist(text)
    top_N = x
    rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
    matplotlib.style.use('ggplot')
    rslt.plot.bar(rot=0)

def wordfreq(text, x):
    word_dist = nltk.FreqDist(text)
    top_N = x
    rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
    return rslt

In [ ]:
# Modify Stopwords to Exclude Class types, suchs as "dress"
new_stop = set(STOPWORDS)
new_stop.update([x.lower() for x in list(data["Class Name"][data["Class Name"].notnull()].unique())]
                + ["dress", "petite"])

# Cloud
cloud(text= data.Title[data.Title.notnull()].astype(str).values,
      title="Titles",
      stopwords= new_stop,
      size = (7,4))

In [ ]:
# Highly Raited
title ="Highly Rated Comments"
temp = data['Review Text'][data.Rating.astype(int) >= 3]

# Modify Stopwords to Exclude Class types, suchs as "dress"
new_stop = set(STOPWORDS)
new_stop.update([x.lower() for x in list(data["Class Name"][data["Class Name"].notnull()].unique())]
                + ["dress", "petite"])

# Cloud
cloud(text= temp.values, title=title,stopwords= new_stop)

# Bar Chart
wordfreq(preprocessing(temp),20).plot.bar(rot=45, legend=False,figsize=(15,5), color='g',
                          title= title)
plt.ylabel("Occurrence Count")
plt.xlabel("Most Frequent Words")
plt.show()

# Low Raited
title ="Most Frequent Words in Low Rated Comments"
temp = data['Review Text'][data.Rating.astype(int) < 3]

# Modify Stopwords to Exclude Class types, suchs as "dress"
new_stop = set(STOPWORDS)
new_stop.update([x.lower() for x in list(data["Class Name"][data["Class Name"].notnull()].unique())]
                + ["dress", "petite", "skirt","shirt"])

# Cloud
cloud(temp.values, title= title, stopwords = new_stop)

In [ ]:
department_set = data["Department Name"][data["Department Name"].notnull()].unique()
division_set = data["Division Name"][data["Division Name"].notnull()].unique()
def cloud_by_category(data, category, subclass):
    """
    Function to create a wordcloud by class and subclass
    Category signifies the column variable
    Subclass refers to the specific value within the categorical variable
    """
    new_stop = set(STOPWORDS)
    new_stop.update([x.lower() for x in list(data["Class Name"][data["Class Name"].notnull()].unique())]
                   + [x.lower() for x in list(data["Department Name"][data["Department Name"].notnull()].unique())]
                   + ["dress", "petite", "jacket","top"])

    # Cloud
    cloud(text= data["Review Text"][data[category]== subclass],
          title="{}".format(subclass),
          stopwords= new_stop,
          size = (10,6))
    
# Plot
cloud_by_category(data, "Division Name", division_set[0])
cloud_by_category(data, "Division Name", division_set[1])
cloud_by_category(data, "Division Name", division_set[2])

In [ ]:
## Helper Functions
from nltk.util import ngrams
from collections import Counter
def get_ngrams(text, n):
    n_grams = ngrams((text), n)
    return [ ' '.join(grams) for grams in n_grams]

def gramfreq(text,n,num):
    # Extracting bigrams
    result = get_ngrams(text,n)
    # Counting bigrams
    result_count = Counter(result)
    # Converting to the result to a data frame
    data = pd.DataFrame.from_dict(result_count, orient='index')
    data = data.rename(columns={'index':'words', 0:'frequency'}) # Renaming index column name
    return df.sort_values(["frequency"],ascending=[0])[:num]

def gram_table(data, gram, length):
    out = pd.DataFrame(index=None)
    for i in gram:
        table = pd.DataFrame(gramfreq(preprocessing(data),i,length).reset_index())
        table.columns = ["{}-Gram".format(i),"Occurrence"]
        out = pd.concat([out, table], axis=1)
    return out

In [ ]:
print("Non-Recommended Items")
gram_table(data= data['Review Text'][data["Recommended IND"].astype(int) == 0], gram=[1,2,3,4,5], length=30)

In [ ]:
print("Recommended Items")
gram_table(data= data['Review Text'][data["Recommended IND"].astype(int) == 1], gram=[1,2,3,4,5], length=30)

In [ ]:
data['tokenized'] = data["Review Text"].astype(str).str.lower() # Turn into lower case text
data['tokenized'] = data.apply(lambda row: tokenizer.tokenize(row['tokenized']), axis=1) # Apply tokenize to each row
data['tokenized'] = data['tokenized'].apply(lambda x: [w for w in x if not w in stop_words]) # Remove stopwords from each row
data['tokenized'] = data['tokenized'].apply(lambda x: [ps.stem(w) for w in x]) # Apply stemming to each row
all_words = nltk.FreqDist(preprocessing(data['Review Text'])) # Calculate word occurrence from whole block of text

vocab_count = 200
word_features= list(all_words.keys())[:vocab_count] # 2000 most recurring unique words
print("Number of words columns (One Hot Encoding): {}".format(len(all_words)))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
import scikitplot as skplt
import eli5

In [ ]:
vect = TfidfVectorizer()
vect.fit(data["Review Text"])
X = vect.transform(data["Review Text"])

In [ ]:
y = data["Recommended IND"].copy()

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.20, random_state=23, stratify=y)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
print("Train Set Accuracy: {}".format(metrics.accuracy_score(model.predict(X_train), y_train)))
print("Train Set ROC: {}\n".format(metrics.roc_auc_score(model.predict(X_train), y_train)))

print("Validation Set Accuracy: {}".format(metrics.accuracy_score(model.predict(X_valid), y_valid)))
print("Validation Set ROC: {}".format(metrics.roc_auc_score(model.predict(X_valid), y_valid)))

In [ ]:
print(metrics.classification_report(model.predict(X_valid), y_valid))

In [ ]:
# Confusion Matrix
skplt.metrics.plot_confusion_matrix(model.predict(X_valid), y_valid, normalize=True)
plt.show()

In [ ]:
target_names = ["Not Recommended","Recommended"]
eli5.show_weights(model, vec=vect, top=100,
                  target_names=target_names)

In [ ]:
for iteration in range(15):
    samp = random.randint(1,data.shape[0])
    print("Real Label: {}".format(data["Recommended IND"].iloc[samp]))
    display(eli5.show_prediction(model,data["Review Text"].iloc[samp], vec=vect,
                         target_names=target_names))